In [1]:
import cdflib
import numpy as np
import plotly.graph_objects as go

import hpe
from constants import *
from utils import occlude_video_region, subclip_video
from visualize import overlay_keypoints

/media/joris/apps/miniconda/envs/openmmlab/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/media/joris/apps/miniconda/envs/openmmlab/lib/python3.8/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [2]:
%reload_ext autoreload
%autoreload 2

subject = 1
vid_name = "Walking 1.54138969.mp4"
vid_name = "Directions 1.54138969.mp4"
video_path = f"{INTERIM_FOLDER}extracted/S{subject}/Videos/{vid_name}"


In [3]:
# video_path = f"{INTERIM_FOLDER}short.mp4"
subclip_start = 0
subclip_end = 1

hpe.predict(
  video_path=video_path,
  subclip=True,
  occlude=True,
  occlusion_proportion_h=0.45,
  subclip_start=subclip_start,
  subclip_end=subclip_end,
)


Subclipping video...
Occluding video region...
Stage 1: 2D pose detection.
Initializing model...
load checkpoint from local path: /media/joris/uca-msc-dsai/research_project/models/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth
load checkpoint from local path: /media/joris/uca-msc-dsai/research_project/models/hrnet_w48_coco_256x192-b9e0b3ab_20200708.pth
Running 2D pose detection inference...
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 50/50, 0.6 task/s, elapsed: 77s, ETA:     0s
Stage 2: 2D-to-3D pose lifting.
Initializing model...
load checkpoint from local path: /media/joris/uca-msc-dsai/research_project/models/videopose_h36m_243frames_fullconv_supervised_cpn_ft-88f5abbb_20210527.pth
Running 2D-to-3D pose lifting inference...
[                                                  ] 0/50, elapsed: 0s, ETA:
counter: 0
[                                  ] 1/50, 2.2 task/s, elapsed: 0s, ETA:    23s
counter: 1
[>                                 ] 2/50, 2.1 task/s, elapsed: 1s, ETA:    22s
counter: 2

In [4]:
kp_path = f"{INTERIM_FOLDER}extracted/S{subject}/MyPoseFeatures/D3_Positions_mono/{vid_name.strip('.mp4')}.cdf"
cdf = cdflib.CDF(kp_path)

poses = cdflib.cdf_to_xarray(kp_path).data_vars["Pose"].values
poses = poses.reshape((-1, 32, 3))
poses = np.array([hpe.convert_keypoint_definition(pose) for pose in poses])
first_pose = int(FPS_H36M * subclip_start)
last_pose = int(FPS_H36M * subclip_end)
# last_pose = FPS_H36M * subclip_start + 1
gt = poses[first_pose:last_pose]
gt.shape


(50, 17, 3)

In [5]:
hpe.results[0]
hpe.pose_lift_results_list[4][0]["keypoints_3d"]

array([[-8.66505783e-04, -9.69548855e-05, -2.48018856e-04,
         1.00000000e+00],
       [ 8.02667961e-02, -7.72723835e-03,  1.43092228e-02,
         1.00000000e+00],
       [ 3.25872283e-03,  4.28616554e-01,  1.15929127e-01,
         1.00000000e+00],
       [ 1.48625329e-01,  9.11728382e-01,  2.45405734e-01,
         1.00000000e+00],
       [-8.08258355e-02,  7.96586089e-03, -1.39733749e-02,
         1.00000000e+00],
       [-5.29629216e-02,  4.57742959e-01,  1.89288720e-01,
         1.00000000e+00],
       [-4.58022542e-02,  8.91989827e-01,  4.21745747e-01,
         1.00000000e+00],
       [ 1.82773918e-02, -1.86679512e-02, -1.04459621e-01,
         1.00000000e+00],
       [ 3.36769149e-02,  8.03468078e-02, -1.22665435e-01,
         1.00000000e+00],
       [-4.36822832e-01,  4.70726907e-01, -1.12107880e-02,
         1.00000000e+00],
       [-3.31214406e-02,  3.78329992e-01,  1.28863081e-02,
         1.00000000e+00],
       [-6.18437938e-02,  3.85737270e-02, -1.84891149e-01,
      

In [6]:
from mmpose.core.evaluation.mesh_eval import compute_similarity_transform

pred = np.array([p[0]["keypoints_3d"][:, :3] for p in hpe.pose_lift_results_list])
# pred = hpe.results[0]["keypoints_3d"]
# pred = pred[None, :]
pred.shape


(50, 17, 3)

In [7]:
pred = np.stack(
  [compute_similarity_transform(pred_i, gt_i) for pred_i, gt_i in zip(pred, gt)]
)

pred.shape

(50, 17, 3)

In [8]:
# pred_dot_pred = np.einsum('nkc,nkc->n', pred, pred)
# pred_dot_gt = np.einsum('nkc,nkc->n', pred, gt)
# pred_dot_pred, pred_dot_gt
# scale_factor = pred_dot_gt / pred_dot_pred
# pred = pred * scale_factor[:, None, None]

error = np.linalg.norm(pred - gt, ord=2, axis=-1).mean()
error

389.38484298373544

In [107]:
n_points = 99
scat_pose = go.Scatter3d(x=gt[0, :n_points, 0], y=gt[0, :n_points, 1], z=gt[0, :n_points, 2], mode="markers", name="GT")
scat_pred = go.Scatter3d(x=pred[0, :n_points, 0], y=pred[0, :n_points, 1], z=pred[0, :n_points, 2], mode="markers", name="Pred")
fig = go.Figure()
fig.add_trace(scat_pose)
fig.add_trace(scat_pred)

fig.update_traces(marker_size=5)
fig


In [105]:
for inf, dat in hpe.info.__dict__.items():
  print(f"\n-----> {inf}")
  print(dat)


-----> _dataset_info
{'dataset_name': 'h36m', 'paper_info': {'author': 'Ionescu, Catalin and Papava, Dragos and Olaru, Vlad and Sminchisescu, Cristian', 'title': 'Human3.6M: Large Scale Datasets and Predictive Methods for 3D Human Sensing in Natural Environments', 'container': 'IEEE Transactions on Pattern Analysis and Machine Intelligence', 'year': '2014', 'homepage': 'http://vision.imar.ro/human3.6m/description.php'}, 'keypoint_info': {0: {'name': 'root', 'id': 0, 'color': [51, 153, 255], 'type': 'lower', 'swap': ''}, 1: {'name': 'right_hip', 'id': 1, 'color': [255, 128, 0], 'type': 'lower', 'swap': 'left_hip'}, 2: {'name': 'right_knee', 'id': 2, 'color': [255, 128, 0], 'type': 'lower', 'swap': 'left_knee'}, 3: {'name': 'right_foot', 'id': 3, 'color': [255, 128, 0], 'type': 'lower', 'swap': 'left_foot'}, 4: {'name': 'left_hip', 'id': 4, 'color': [0, 255, 0], 'type': 'lower', 'swap': 'right_hip'}, 5: {'name': 'left_knee', 'id': 5, 'color': [0, 255, 0], 'type': 'lower', 'swap': 'right

In [114]:
hpe.results[0]

{'track_id': 0,
 'keypoints': array([[4.77087494e+02, 4.52368164e+02, 7.68301964e-01],
        [4.97203064e+02, 4.53804993e+02, 7.71838546e-01],
        [4.82834778e+02, 5.37140869e+02, 8.98325205e-01],
        [5.00076721e+02, 6.17603088e+02, 8.99763048e-01],
        [4.56971924e+02, 4.50931335e+02, 7.64765382e-01],
        [4.59845581e+02, 5.42888184e+02, 9.17850852e-01],
        [4.65592896e+02, 6.23350403e+02, 8.92481506e-01],
        [4.79961151e+02, 4.04952942e+02, 8.18031847e-01],
        [4.82834808e+02, 3.57537720e+02, 8.67761731e-01],
        [4.42603699e+02, 3.56100891e+02, 1.22218594e-01],
        [4.41166870e+02, 3.56100891e+02, 1.22560464e-01],
        [4.51224670e+02, 3.58974548e+02, 8.67326021e-01],
        [4.31109070e+02, 4.19321228e+02, 9.00093913e-01],
        [4.13867188e+02, 4.59552307e+02, 8.58214140e-01],
        [5.14444946e+02, 3.56100891e+02, 8.68197441e-01],
        [5.20192261e+02, 4.16447571e+02, 9.41408992e-01],
        [5.17318604e+02, 4.59552307e+02, 8.

In [107]:
import cv2
import plotly.express as px

short = f"{INTERIM_FOLDER}short.mp4"

cap = cv2.VideoCapture(short)
success = True

# while success:
success, frame_bgr = cap.read()
frame = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)

res = hpe.results[0]

for kp in res["keypoints"]:
  cv2.circle(
    img=frame,
    center=[int(c) for c in kp[:2]],
    radius=4,
    color=(0, 255, 0),
    thickness=-1,
  )

cv2.rectangle(
  img=frame,
  pt1=[int(p) for p in res["bbox"][:2]],
  pt2=[int(p) for p in res["bbox"][2:4]],
  color=(0, 255, 0),
  thickness=2,
)

px.imshow(frame).show()

# break


In [48]:
res

{'track_id': 0,
 'keypoints': array([[4.8700259e+02, 5.0373004e+02, 3.7270606e-02],
        [5.0395004e+02, 5.0779742e+02, 2.4238542e-02],
        [5.0937320e+02, 5.3898071e+02, 6.6548681e-01],
        [4.9852682e+02, 6.1761682e+02, 7.5340807e-01],
        [4.7005515e+02, 4.9966266e+02, 5.0302669e-02],
        [4.7276675e+02, 5.4169232e+02, 6.6259027e-01],
        [4.6869937e+02, 6.2032843e+02, 7.6681387e-01],
        [4.8530786e+02, 5.0576373e+02, 6.5473475e-02],
        [4.8361310e+02, 5.0779742e+02, 9.3676344e-02],
        [4.6869937e+02, 5.5118286e+02, 4.0922124e-02],
        [4.7412253e+02, 5.4101440e+02, 1.6487775e-02],
        [4.6869937e+02, 5.0779742e+02, 1.1031245e-01],
        [4.7141098e+02, 5.4304810e+02, 2.6294208e-01],
        [4.6869937e+02, 6.2032843e+02, 3.4563774e-01],
        [4.9852682e+02, 5.0779742e+02, 7.7040233e-02],
        [5.0937320e+02, 5.3626910e+02, 2.4599938e-01],
        [4.9852682e+02, 6.1626105e+02, 3.0636129e-01]], dtype=float32),
 'keypoints_3d': ar

In [64]:
video_path=f"{RAW_FOLDER}s1/S1/Videos/Directions 1.54138969.mp4"
kp_path=f"{RAW_FOLDER}s1/Poses_D2_Positions_S1/S1/MyPoseFeatures/D2_Positions/Directions 1.54138969.mp4"
figs = overlay_keypoints(
  video_path=video_path,
  kp_path=kp_path,
  max_frame=1000,
  show_nth_frame=400,
)
for fig in figs:
  fig.show()
